# Snowflake Dynamic Tables - GitHub Direct Loading

Load data **directly from GitHub** without local downloads!

## Features
- ✅ No local file downloads needed
- ✅ Loads CSV files directly from GitHub raw URLs
- ✅ Creates Dynamic Tables (Bronze, Silver, Gold)
- ✅ Processes 4 incremental batches

**Repository:** https://github.com/NickAkincilar/Snowflake_DT_vs_DLT/

## Prerequisites
- Snowflake account with ACCOUNTADMIN role
- Network access from Snowflake to GitHub (enabled by default)
- Python 3.8+
- snowflake-snowpark-python package

In [34]:
# Create Session 
from snowflake.snowpark import Session


session = Session.builder.getOrCreate()

session.use_role("ACCOUNTADMIN")



print("✅ Session created")
print(f"   Database: {session.get_current_database()}")
print(f"   Schema: {session.get_current_schema()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Role: {session.get_current_role()}")

In [35]:
# Create database and schemas
session.sql("""
CREATE DATABASE IF NOT EXISTS TPCH_DEMO
  COMMENT = 'TPCH Demo - Dynamic Tables vs DLT'
""").collect()

session.use_database("TPCH_DEMO")

for schema in ["RAW"]:
    session.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}").collect()
    print(f"✅ Schema {schema}")

for schema in [ "BRONZE", "SILVER", "GOLD", "STAGING"]:
    session.sql(f"CREATE OR REPLACE SCHEMA {schema}").collect()
    print(f"✅ Schema {schema}")

# Create warehouse
session.sql("""
CREATE OR REPLACE WAREHOUSE TPCH_DT_WH
  WAREHOUSE_SIZE = 'XSMALL'
  RESOURCE_CONSTRAINT = STANDARD_GEN_2
  AUTO_SUSPEND = 20
  AUTO_RESUME = TRUE
""").collect()

session.use_warehouse("TPCH_DT_WH")
session.use_schema("TPCH_DEMO.RAW")
print("\n✅ Environment ready")

In [36]:
# Create Raw layer using Standard tables
session.use_warehouse("COMPUTE_WH")

session.sql("""
CREATE OR REPLACE TABLE RAW_NATION AS
SELECT 
  N_NATIONKEY,
  N_NAME,
  N_REGIONKEY,
  N_COMMENT
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.NATION
""").collect()

print(f"✅ NATION: {session.table('RAW_NATION').count():,} rows")

# Load REGION table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
session.sql("""
CREATE OR REPLACE TABLE RAW_REGION AS
SELECT 
  R_REGIONKEY,
  R_NAME,
  R_COMMENT
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.REGION
""").collect()

print(f"✅ REGION: {session.table('RAW_REGION').count():,} rows")

# Load PART table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
session.sql("""
CREATE OR REPLACE TABLE RAW_PART AS
SELECT 
  P_PARTKEY,
  P_NAME,
  P_MFGR,
  P_BRAND,
  P_TYPE,
  P_SIZE,
  P_CONTAINER,
  P_RETAILPRICE,
  P_COMMENT,
  CURRENT_TIMESTAMP() AS P_LOAD_TIMESTAMP
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.PART
""").collect()

print(f"✅ PART: {session.table('RAW_PART').count():,} rows")


# Load SUPPLIER table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
session.sql("""
CREATE OR REPLACE TABLE RAW_SUPPLIER AS
SELECT 
  S_SUPPKEY,
  S_NAME,
  S_ADDRESS,
  S_NATIONKEY,
  S_PHONE,
  S_ACCTBAL,
  S_COMMENT,
  CURRENT_TIMESTAMP() AS S_LOAD_TIMESTAMP
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.SUPPLIER
""").collect()

print(f"✅ SUPPLIER: {session.table('RAW_SUPPLIER').count():,} rows")


# Load CUSTOMER table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
session.sql("""
CREATE OR REPLACE TABLE RAW_CUSTOMER AS
SELECT 
  C_CUSTKEY,
  C_NAME,
  C_ADDRESS,
  C_NATIONKEY,
  C_PHONE,
  C_ACCTBAL,
  C_MKTSEGMENT,
  C_COMMENT,
  CURRENT_TIMESTAMP() AS LOAD_TIMESTAMP
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.CUSTOMER
""").collect()

print(f"✅ CUSTOMER: {session.table('RAW_CUSTOMER').count():,} rows")


# Load ORDERS table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
session.sql("""
CREATE OR REPLACE TABLE RAW_ORDERS AS
SELECT 
  O_ORDERKEY,
  O_CUSTKEY,
  O_ORDERSTATUS,
  O_TOTALPRICE,
  O_ORDERDATE,
  O_ORDERPRIORITY,
  O_CLERK,
  O_SHIPPRIORITY,
  O_COMMENT,
  CURRENT_TIMESTAMP() AS LOAD_TIMESTAMP
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS
""").collect()

print(f"✅ ORDERS: {session.table('RAW_ORDERS').count():,} rows")


# Load LINEITEM table from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10
print("Loading lineitem data from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10...")

session.sql("""
CREATE OR REPLACE TABLE RAW_LINEITEM AS
SELECT 
  L_ORDERKEY,
  L_PARTKEY,
  L_SUPPKEY,
  L_LINENUMBER,
  L_QUANTITY,
  L_EXTENDEDPRICE,
  L_DISCOUNT,
  L_TAX,
  L_RETURNFLAG,
  L_LINESTATUS,
  L_SHIPDATE,
  L_COMMITDATE,
  L_RECEIPTDATE,
  L_SHIPINSTRUCT,
  L_SHIPMODE,
  L_COMMENT,
  CURRENT_TIMESTAMP() AS LOAD_TIMESTAMP
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM
""").collect()

lineitem_count = session.table('RAW_LINEITEM').count()
print(f"\n✅ LINEITEM: {lineitem_count:,} rows from TPCH_SF10")

In [37]:
# Create Staging tables for 4 batches of updates  
session.use_schema('STAGING')

# Create Staging Tables V2 with 4 batches - each with different row counts
# Customers & Orders have both INSERTs and UPDATEs per batch
print("Creating Staging tables for incremental processing...\n")

# STG_CUSTOMER: 4 batches with INSERTs and UPDATEs
print("Creating STG_CUSTOMER with 4 batches (INSERTs + UPDATEs)...")
session.sql("""
CREATE OR REPLACE TABLE STG_CUSTOMER AS
WITH max_key AS (
  SELECT MAX(C_CUSTKEY) AS max_custkey
  FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.CUSTOMER
),
sample_customers AS (
  SELECT
    C_CUSTKEY,
    C_NAME,
    C_ADDRESS,
    C_NATIONKEY,
    C_PHONE,
    C_ACCTBAL,
    C_MKTSEGMENT,
    C_COMMENT,
    ROW_NUMBER() OVER (ORDER BY C_CUSTKEY) AS rn
  FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.CUSTOMER
  WHERE C_CUSTKEY <= 700
),
inserts AS (
  SELECT
    rn AS ROW_ID,
    'INSERT' AS CHANGE_TYPE,
    CASE 
      WHEN rn <= 100 THEN 'batch_001'
      WHEN rn <= 250 THEN 'batch_002'
      WHEN rn <= 450 THEN 'batch_003'
      ELSE 'batch_004'
    END AS BATCH_NUMBER,
    (SELECT max_custkey FROM max_key) + rn AS C_CUSTKEY,
    C_NAME,
    C_ADDRESS,
    C_NATIONKEY,
    C_PHONE,
    C_ACCTBAL,
    C_MKTSEGMENT,
    C_COMMENT,
    CURRENT_TIMESTAMP() AS STAGED_AT
  FROM sample_customers
),
updates AS (
  SELECT
    rn + 10000 AS ROW_ID,
    'UPDATE' AS CHANGE_TYPE,
    CASE 
      WHEN rn <= 100 THEN 'batch_001'
      WHEN rn <= 250 THEN 'batch_002'
      WHEN rn <= 450 THEN 'batch_003'
      ELSE 'batch_004'
    END AS BATCH_NUMBER,
    C_CUSTKEY,
    SUBSTR(C_NAME, 1, LENGTH(C_NAME) - 2) || '_U' AS C_NAME,
    SUBSTR(C_ADDRESS, 1, LENGTH(C_ADDRESS) - 3) || '_V2' AS C_ADDRESS,
    C_NATIONKEY,
    C_PHONE,
    C_ACCTBAL * 1.1 AS C_ACCTBAL,
    C_MKTSEGMENT,
    'Updated in ' || CASE 
      WHEN rn <= 100 THEN 'batch_001'
      WHEN rn <= 250 THEN 'batch_002'
      WHEN rn <= 450 THEN 'batch_003'
      ELSE 'batch_004'
    END AS C_COMMENT,
    CURRENT_TIMESTAMP() AS STAGED_AT
  FROM sample_customers
  WHERE MOD(rn, 4) = 0
)
SELECT * FROM inserts
UNION ALL
SELECT * FROM updates
""").collect()

cust_insert_count = session.sql("SELECT COUNT(*) FROM STG_CUSTOMER WHERE CHANGE_TYPE = 'INSERT'").collect()[0][0]
cust_update_count = session.sql("SELECT COUNT(*) FROM STG_CUSTOMER WHERE CHANGE_TYPE = 'UPDATE'").collect()[0][0]
print(f"  ✅ Total: {cust_insert_count} INSERTs, {cust_update_count} UPDATEs")

# STG_ORDERS: 4 batches (300, 500, 700, 900 rows)
print("Creating STG_ORDERS with 4 batches (INSERTs + UPDATEs)...")
session.sql("""
CREATE OR REPLACE TABLE STG_ORDERS AS
WITH max_keys AS (
  SELECT 
    MAX(O_ORDERKEY) AS max_orderkey,
    (SELECT MAX(C_CUSTKEY) FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.CUSTOMER) AS max_custkey
  FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS
),
sample_orders AS (
  SELECT
    O_ORDERKEY,
    O_ORDERSTATUS,
    O_CUSTKEY,
    O_TOTALPRICE,
    O_ORDERDATE,
    O_ORDERPRIORITY,
    O_CLERK,
    O_SHIPPRIORITY,
    O_COMMENT,
    ROW_NUMBER() OVER (ORDER BY O_ORDERKEY) AS rn
  FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS
  WHERE O_ORDERKEY <= 2400
),
inserts AS (
  SELECT
    rn AS ROW_ID,
    'INSERT' AS CHANGE_TYPE,
    CASE 
      WHEN rn <= 300 THEN 'batch_001'
      WHEN rn <= 800 THEN 'batch_002'
      WHEN rn <= 1500 THEN 'batch_003'
      ELSE 'batch_004'
    END AS BATCH_NUMBER,
    (SELECT max_orderkey FROM max_keys) + rn AS O_ORDERKEY,
    CASE 
      WHEN O_CUSTKEY <= 700 THEN (SELECT max_custkey FROM max_keys) + O_CUSTKEY
      ELSE O_CUSTKEY
    END AS O_CUSTKEY,
    O_ORDERSTATUS,
    O_TOTALPRICE,
    O_ORDERDATE,
    O_ORDERPRIORITY,
    O_CLERK,
    O_SHIPPRIORITY,
    O_COMMENT,
    CURRENT_TIMESTAMP() AS STAGED_AT
  FROM sample_orders
),
updates AS (
  SELECT
    rn + 20000 AS ROW_ID,
    'UPDATE' AS CHANGE_TYPE,
    CASE 
      WHEN rn <= 300 THEN 'batch_001'
      WHEN rn <= 800 THEN 'batch_002'
      WHEN rn <= 1500 THEN 'batch_003'
      ELSE 'batch_004'
    END AS BATCH_NUMBER,
    O_ORDERKEY,  -- Use existing order ids from sample_orders
    O_CUSTKEY,
    O_ORDERSTATUS,
    O_TOTALPRICE * 1.05 AS O_TOTALPRICE,
    O_ORDERDATE,
    O_ORDERPRIORITY,
    O_CLERK,
    O_SHIPPRIORITY,
    'Updated in batch' AS O_COMMENT,
    CURRENT_TIMESTAMP() AS STAGED_AT
  FROM sample_orders
  WHERE MOD(rn, 5) = 0
)
SELECT * FROM inserts
UNION ALL
SELECT * FROM updates
""").collect()
print(f"  ✅ Batch 1: 300 orders, Batch 2: 500, Batch 3: 700, Batch 4: 900")

# STG_LINEITEM: 4 batches (1000, 1500, 2000, 2500 rows)
print("Creating STG_LINEITEM with 4 batches...")
# STG_LINEITEM: Maps directly to O_ORDERKEYs created in STG_ORDERS
print("Creating STG_LINEITEM with 4 batches...")
session.sql("""
CREATE OR REPLACE TABLE STG_LINEITEM AS
WITH stg_order_keys AS (
    -- Get all INSERT order keys from STG_ORDERS to ensure referential integrity
    SELECT O_ORDERKEY, BATCH_NUMBER
    FROM STG_ORDERS
    WHERE CHANGE_TYPE = 'INSERT'
),
sample_lineitems AS (
    -- Get a pool of line item details to reuse
    -- We use ROW_NUMBER to pick specific items per order later
    SELECT
        L_PARTKEY, L_SUPPKEY, L_LINENUMBER, L_QUANTITY, L_EXTENDEDPRICE,
        L_DISCOUNT, L_TAX, L_RETURNFLAG, L_LINESTATUS, L_SHIPDATE,
        L_COMMITDATE, L_RECEIPTDATE, L_SHIPINSTRUCT, L_SHIPMODE, L_COMMENT,
        ROW_NUMBER() OVER (ORDER BY L_ORDERKEY, L_LINENUMBER) as row_num
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM
    LIMIT 10000 
)
SELECT
    ROW_NUMBER() OVER (ORDER BY ok.BATCH_NUMBER, ok.O_ORDERKEY) AS ROW_ID,
    'INSERT' AS CHANGE_TYPE,
    ok.BATCH_NUMBER,        -- Keep it in the same batch as the parent Order
    ok.O_ORDERKEY AS L_ORDERKEY,
    sl.L_PARTKEY,
    sl.L_SUPPKEY,
    sl.L_LINENUMBER,
    sl.L_QUANTITY,
    sl.L_EXTENDEDPRICE,
    sl.L_DISCOUNT,
    sl.L_TAX,
    sl.L_RETURNFLAG,
    sl.L_LINESTATUS,
    sl.L_SHIPDATE,
    sl.L_COMMITDATE,
    sl.L_RECEIPTDATE,
    sl.L_SHIPINSTRUCT,
    sl.L_SHIPMODE,
    sl.L_COMMENT,
    CURRENT_TIMESTAMP() AS STAGED_AT
FROM stg_order_keys ok
-- Join to sample_lineitems to give each order 1-3 line items
JOIN sample_lineitems sl 
  ON sl.row_num % 2400 = ok.O_ORDERKEY % 2400 
""").collect()

line_count = session.sql("SELECT COUNT(*) FROM STG_LINEITEM").collect()[0][0]
print(f"  ✅ Total: {line_count} INSERTs mapped to STG_ORDERS")

print("\n✅ All staging tables created with 4 batches each!")


In [38]:
# Create Bronze Layer Dynamic Tables
session.use_schema("TPCH_DEMO.BRONZE")

print("Creating Bronze Layer Dynamic Tables...")

# Bronze Customer Dynamic Table
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE BRONZE_CUSTOMER
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  C_CUSTKEY,
  C_NAME,
  C_ADDRESS,
  C_NATIONKEY,
  C_PHONE,
  C_ACCTBAL,
  C_MKTSEGMENT,
  C_COMMENT
FROM RAW.RAW_CUSTOMER
WHERE C_CUSTKEY IS NOT NULL
""").collect()
print("✅ Bronze Customer Dynamic Table created")

# Bronze Orders Dynamic Table
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE BRONZE_ORDERS
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  O_ORDERKEY,
  O_CUSTKEY,
  O_ORDERSTATUS,
  O_TOTALPRICE,
  O_ORDERDATE,
  O_ORDERPRIORITY,
  O_CLERK,
  O_SHIPPRIORITY,
  O_COMMENT
FROM RAW.RAW_ORDERS
WHERE O_ORDERKEY IS NOT NULL
""").collect()
print("✅ Bronze Orders Dynamic Table created")

# Bronze Lineitem Dynamic Table
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE BRONZE_LINEITEM
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  L_ORDERKEY,
  L_PARTKEY,
  L_SUPPKEY,
  L_LINENUMBER,
  L_QUANTITY,
  L_EXTENDEDPRICE,
  L_DISCOUNT,
  L_TAX,
  L_RETURNFLAG,
  L_LINESTATUS,
  L_SHIPDATE,
  L_COMMITDATE,
  L_RECEIPTDATE,
  L_SHIPINSTRUCT,
  L_SHIPMODE,
  L_COMMENT
FROM RAW.RAW_LINEITEM
WHERE L_ORDERKEY IS NOT NULL AND L_LINENUMBER IS NOT NULL
""").collect()
print("✅ Bronze Lineitem Dynamic Table created")

print("\n✅ Bronze Layer Dynamic Tables created successfully")


In [39]:
# Create Silver Layer Dynamic Tables with Business Transformations
session.use_schema("SILVER")

print("Creating Silver Layer Dynamic Tables with business transformations...")

# Silver Customer Dynamic Table - with data quality and derived columns
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE SILVER_CUSTOMER
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  C_CUSTKEY,
  UPPER(TRIM(C_NAME)) AS C_NAME,
  TRIM(C_ADDRESS) AS C_ADDRESS,
  C_NATIONKEY,
  REGEXP_REPLACE(C_PHONE, '[^0-9-]', '') AS C_PHONE,
  C_ACCTBAL,
  UPPER(TRIM(C_MKTSEGMENT)) AS C_MKTSEGMENT,
  TRIM(C_COMMENT) AS C_COMMENT,
  CASE 
    WHEN C_ACCTBAL < 0 THEN 'NEGATIVE'
    WHEN C_ACCTBAL = 0 THEN 'ZERO'
    WHEN C_ACCTBAL BETWEEN 0 AND 1000 THEN 'LOW'
    WHEN C_ACCTBAL BETWEEN 1000 AND 5000 THEN 'MEDIUM'
    WHEN C_ACCTBAL BETWEEN 5000 AND 10000 THEN 'HIGH'
    ELSE 'VERY_HIGH'
  END AS ACCOUNT_BALANCE_TIER
FROM BRONZE.BRONZE_CUSTOMER
""").collect()
print("✅ Silver Customer Dynamic Table created with transformations")

# Silver Orders Dynamic Table - with derived metrics and status categorization
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE SILVER_ORDERS
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  O_ORDERKEY,
  O_CUSTKEY,
  UPPER(TRIM(O_ORDERSTATUS)) AS O_ORDERSTATUS,
  CASE UPPER(TRIM(O_ORDERSTATUS))
    WHEN 'O' THEN 'OPEN'
    WHEN 'F' THEN 'FULFILLED'
    WHEN 'P' THEN 'PENDING'
    ELSE 'UNKNOWN'
  END AS ORDER_STATUS_DESC,
  O_TOTALPRICE,
  CASE 
    WHEN O_TOTALPRICE < 1000 THEN 'SMALL'
    WHEN O_TOTALPRICE BETWEEN 1000 AND 10000 THEN 'MEDIUM'
    WHEN O_TOTALPRICE BETWEEN 10000 AND 50000 THEN 'LARGE'
    ELSE 'ENTERPRISE'
  END AS ORDER_SIZE_CATEGORY,
  O_ORDERDATE,
  YEAR(O_ORDERDATE) AS ORDER_YEAR,
  QUARTER(O_ORDERDATE) AS ORDER_QUARTER,
  MONTH(O_ORDERDATE) AS ORDER_MONTH,
  DAYOFWEEK(O_ORDERDATE) AS ORDER_DAY_OF_WEEK,
  UPPER(TRIM(O_ORDERPRIORITY)) AS O_ORDERPRIORITY,
  CASE UPPER(TRIM(O_ORDERPRIORITY))
    WHEN '1-URGENT' THEN 1
    WHEN '2-HIGH' THEN 2
    WHEN '3-MEDIUM' THEN 3
    WHEN '4-NOT SPECIFIED' THEN 4
    WHEN '5-LOW' THEN 5
    ELSE 99
  END AS PRIORITY_RANK,
  UPPER(TRIM(O_CLERK)) AS O_CLERK,
  O_SHIPPRIORITY,
  TRIM(O_COMMENT) AS O_COMMENT
FROM BRONZE.BRONZE_ORDERS
""").collect()
print("✅ Silver Orders Dynamic Table created with transformations")

# Silver Lineitem Dynamic Table - with calculated fields and business logic
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE SILVER_LINEITEM
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  L_ORDERKEY,
  L_PARTKEY,
  L_SUPPKEY,
  L_LINENUMBER,
  L_QUANTITY,
  L_EXTENDEDPRICE,
  L_DISCOUNT,
  L_TAX,
  UPPER(TRIM(L_RETURNFLAG)) AS L_RETURNFLAG,
  UPPER(TRIM(L_LINESTATUS)) AS L_LINESTATUS,
  CASE UPPER(TRIM(L_LINESTATUS))
    WHEN 'O' THEN 'OPEN'
    WHEN 'F' THEN 'FULFILLED'
    ELSE 'UNKNOWN'
  END AS LINE_STATUS_DESC,
  L_SHIPDATE,
  L_COMMITDATE,
  L_RECEIPTDATE,
  UPPER(TRIM(L_SHIPINSTRUCT)) AS L_SHIPINSTRUCT,
  UPPER(TRIM(L_SHIPMODE)) AS L_SHIPMODE,
  TRIM(L_COMMENT) AS L_COMMENT,
  L_EXTENDEDPRICE * (1 - L_DISCOUNT) AS DISCOUNTED_PRICE,
  L_EXTENDEDPRICE * (1 - L_DISCOUNT) * (1 + L_TAX) AS FINAL_PRICE,
  L_EXTENDEDPRICE * L_DISCOUNT AS DISCOUNT_AMOUNT,
  L_EXTENDEDPRICE * (1 - L_DISCOUNT) * L_TAX AS TAX_AMOUNT,
  ROUND((L_DISCOUNT * 100), 2) AS DISCOUNT_PERCENT,
  ROUND((L_TAX * 100), 2) AS TAX_PERCENT,
  CASE 
    WHEN L_DISCOUNT = 0 THEN 'NO_DISCOUNT'
    WHEN L_DISCOUNT <= 0.05 THEN 'LOW_DISCOUNT'
    WHEN L_DISCOUNT <= 0.10 THEN 'MEDIUM_DISCOUNT'
    ELSE 'HIGH_DISCOUNT'
  END AS DISCOUNT_TIER,
  DATEDIFF(DAY, L_SHIPDATE, L_RECEIPTDATE) AS SHIPPING_DAYS,
  DATEDIFF(DAY, L_COMMITDATE, L_RECEIPTDATE) AS DAYS_FROM_COMMIT,
  CASE 
    WHEN L_RECEIPTDATE <= L_COMMITDATE THEN 'ON_TIME'
    WHEN DATEDIFF(DAY, L_COMMITDATE, L_RECEIPTDATE) <= 3 THEN 'SLIGHTLY_LATE'
    ELSE 'LATE'
  END AS DELIVERY_PERFORMANCE
FROM BRONZE.BRONZE_LINEITEM
""").collect()
print("✅ Silver Lineitem Dynamic Table created with transformations")

print("\n✅ Silver Layer Dynamic Tables created successfully with business transformations!")
print("   - Data standardization (UPPER, TRIM)")
print("   - Derived categories and tiers")
print("   - Calculated financial metrics")
print("   - Date/time components")
print("   - Performance indicators")


In [40]:
# Create Gold Layer Dynamic Tables for Advanced Analytics
session.use_schema("TPCH_DEMO.GOLD")

print("Creating Gold Layer Dynamic Tables with advanced analytics...")

# Gold Customer Orders Summary - Enhanced with Silver transformations
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE GOLD_CUSTOMER_ORDERS_SUMMARY
TARGET_LAG = '1 minute'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  c.C_CUSTKEY,
  c.C_NAME,
  c.C_MKTSEGMENT,
  c.ACCOUNT_BALANCE_TIER,
  c.C_NATIONKEY,
  COUNT(DISTINCT o.O_ORDERKEY) AS TOTAL_ORDERS,
  COUNT(DISTINCT CASE WHEN o.ORDER_STATUS_DESC = 'FULFILLED' THEN o.O_ORDERKEY END) AS FULFILLED_ORDERS,
  COUNT(DISTINCT CASE WHEN o.ORDER_STATUS_DESC = 'OPEN' THEN o.O_ORDERKEY END) AS OPEN_ORDERS,
  SUM(o.O_TOTALPRICE) AS TOTAL_REVENUE,
  AVG(o.O_TOTALPRICE) AS AVG_ORDER_VALUE,
  MIN(o.O_TOTALPRICE) AS MIN_ORDER_VALUE,
  MAX(o.O_TOTALPRICE) AS MAX_ORDER_VALUE,
  MIN(o.O_ORDERDATE) AS FIRST_ORDER_DATE,
  MAX(o.O_ORDERDATE) AS LAST_ORDER_DATE,
  DATEDIFF(DAY, MIN(o.O_ORDERDATE), MAX(o.O_ORDERDATE)) AS CUSTOMER_LIFETIME_DAYS,
  COUNT(DISTINCT o.ORDER_YEAR) AS ACTIVE_YEARS,
  COUNT(DISTINCT CASE WHEN o.PRIORITY_RANK <= 2 THEN o.O_ORDERKEY END) AS HIGH_PRIORITY_ORDERS,
  ROUND(AVG(o.PRIORITY_RANK), 2) AS AVG_PRIORITY_RANK,
  MAX(o.ORDER_SIZE_CATEGORY) AS LARGEST_ORDER_CATEGORY,
  CASE 
    WHEN COUNT(DISTINCT o.O_ORDERKEY) >= 10 THEN 'VIP'
    WHEN COUNT(DISTINCT o.O_ORDERKEY) >= 5 THEN 'FREQUENT'
    WHEN COUNT(DISTINCT o.O_ORDERKEY) >= 2 THEN 'REGULAR'
    ELSE 'OCCASIONAL'
  END AS CUSTOMER_SEGMENT
FROM SILVER.SILVER_CUSTOMER c
LEFT JOIN SILVER.SILVER_ORDERS o ON c.C_CUSTKEY = o.O_CUSTKEY
GROUP BY c.C_CUSTKEY, c.C_NAME, c.C_MKTSEGMENT, c.ACCOUNT_BALANCE_TIER, c.C_NATIONKEY
""").collect()
print("✅ Gold Customer Orders Summary created with advanced metrics")

# Gold Order Line Items Summary - Enhanced with calculated fields from Silver
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE GOLD_ORDER_LINEITEM_SUMMARY
TARGET_LAG = '1 minute'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  o.O_ORDERKEY,
  o.O_CUSTKEY,
  o.O_ORDERSTATUS,
  o.ORDER_STATUS_DESC,
  o.O_ORDERDATE,
  o.ORDER_YEAR,
  o.ORDER_QUARTER,
  o.ORDER_MONTH,
  o.ORDER_SIZE_CATEGORY,
  o.PRIORITY_RANK,
  COUNT(l.L_LINENUMBER) AS TOTAL_LINE_ITEMS,
  SUM(l.L_QUANTITY) AS TOTAL_QUANTITY,
  SUM(l.L_EXTENDEDPRICE) AS TOTAL_EXTENDED_PRICE,
  SUM(l.DISCOUNTED_PRICE) AS TOTAL_DISCOUNTED_PRICE,
  SUM(l.FINAL_PRICE) AS TOTAL_FINAL_PRICE,
  SUM(l.DISCOUNT_AMOUNT) AS TOTAL_DISCOUNT_AMOUNT,
  SUM(l.TAX_AMOUNT) AS TOTAL_TAX_AMOUNT,
  ROUND(AVG(l.DISCOUNT_PERCENT), 2) AS AVG_DISCOUNT_PERCENT,
  ROUND(AVG(l.TAX_PERCENT), 2) AS AVG_TAX_PERCENT,
  ROUND(AVG(l.SHIPPING_DAYS), 1) AS AVG_SHIPPING_DAYS,
  COUNT(CASE WHEN l.DELIVERY_PERFORMANCE = 'ON_TIME' THEN 1 END) AS ON_TIME_DELIVERIES,
  COUNT(CASE WHEN l.DELIVERY_PERFORMANCE = 'LATE' THEN 1 END) AS LATE_DELIVERIES,
  ROUND(COUNT(CASE WHEN l.DELIVERY_PERFORMANCE = 'ON_TIME' THEN 1 END) * 100.0 / NULLIF(COUNT(l.L_LINENUMBER), 0), 2) AS ON_TIME_DELIVERY_RATE,
  COUNT(CASE WHEN l.DISCOUNT_TIER = 'HIGH_DISCOUNT' THEN 1 END) AS HIGH_DISCOUNT_ITEMS,
  COUNT(CASE WHEN l.L_RETURNFLAG = 'R' THEN 1 END) AS RETURNED_ITEMS,
  MAX(l.L_SHIPMODE) AS PRIMARY_SHIP_MODE
FROM SILVER.SILVER_ORDERS o
LEFT JOIN SILVER.SILVER_LINEITEM l ON o.O_ORDERKEY = l.L_ORDERKEY 
GROUP BY 
  o.O_ORDERKEY, o.O_CUSTKEY, o.O_ORDERSTATUS, o.ORDER_STATUS_DESC,
  o.O_ORDERDATE, o.ORDER_YEAR, o.ORDER_QUARTER, o.ORDER_MONTH,
  o.ORDER_SIZE_CATEGORY, o.PRIORITY_RANK
""").collect()
print("✅ Gold Order LineItem Summary created with delivery & discount analytics")

# Gold Daily Sales Summary - Enhanced with Silver transformations
session.sql("""
CREATE OR REPLACE DYNAMIC TABLE GOLD_DAILY_SALES_SUMMARY
TARGET_LAG = '1 minute'
REFRESH_MODE = INCREMENTAL 
WAREHOUSE = TPCH_DT_WH
AS
SELECT
  o.O_ORDERDATE AS SALE_DATE,
  o.ORDER_YEAR,
  o.ORDER_QUARTER,
  o.ORDER_MONTH,
  o.ORDER_DAY_OF_WEEK,
  o.O_ORDERSTATUS,
  o.ORDER_STATUS_DESC,
  COUNT(DISTINCT o.O_ORDERKEY) AS TOTAL_ORDERS,
  COUNT(DISTINCT o.O_CUSTKEY) AS UNIQUE_CUSTOMERS,
  SUM(l.FINAL_PRICE) AS TOTAL_REVENUE,
  SUM(l.DISCOUNTED_PRICE) AS TOTAL_DISCOUNTED_REVENUE,
  SUM(l.DISCOUNT_AMOUNT) AS TOTAL_DISCOUNTS_GIVEN,
  SUM(l.TAX_AMOUNT) AS TOTAL_TAX_COLLECTED,
  AVG(l.FINAL_PRICE) AS AVG_LINE_ITEM_REVENUE,
  SUM(l.L_QUANTITY) AS TOTAL_QUANTITY_SOLD,
  COUNT(l.L_LINENUMBER) AS TOTAL_LINE_ITEMS,
  ROUND(AVG(l.DISCOUNT_PERCENT), 2) AS AVG_DISCOUNT_PERCENT,
  COUNT(CASE WHEN o.ORDER_SIZE_CATEGORY = 'ENTERPRISE' THEN 1 END) AS ENTERPRISE_ORDERS,
  COUNT(CASE WHEN o.ORDER_SIZE_CATEGORY = 'LARGE' THEN 1 END) AS LARGE_ORDERS,
  COUNT(CASE WHEN o.ORDER_SIZE_CATEGORY = 'MEDIUM' THEN 1 END) AS MEDIUM_ORDERS,
  COUNT(CASE WHEN o.ORDER_SIZE_CATEGORY = 'SMALL' THEN 1 END) AS SMALL_ORDERS,
  COUNT(CASE WHEN o.PRIORITY_RANK = 1 THEN 1 END) AS URGENT_ORDERS,
  ROUND(AVG(l.SHIPPING_DAYS), 1) AS AVG_SHIPPING_DAYS,
  COUNT(CASE WHEN l.DELIVERY_PERFORMANCE = 'ON_TIME' THEN 1 END) AS ON_TIME_DELIVERIES,
  ROUND(COUNT(CASE WHEN l.DELIVERY_PERFORMANCE = 'ON_TIME' THEN 1 END) * 100.0 / NULLIF(COUNT(l.L_LINENUMBER), 0), 2) AS ON_TIME_DELIVERY_RATE
FROM SILVER.SILVER_ORDERS o
LEFT JOIN SILVER.SILVER_LINEITEM l ON o.O_ORDERKEY = l.L_ORDERKEY 
GROUP BY 
  o.O_ORDERDATE, o.ORDER_YEAR, o.ORDER_QUARTER, o.ORDER_MONTH, 
  o.ORDER_DAY_OF_WEEK, o.O_ORDERSTATUS, o.ORDER_STATUS_DESC
""").collect()
print("✅ Gold Daily Sales Summary created with comprehensive KPIs")

print("\n✅ Gold Layer Dynamic Tables created successfully with advanced analytics!")
print("   - Customer segmentation and lifetime metrics")
print("   - Order fulfillment and priority analysis")
print("   - Delivery performance tracking")
print("   - Discount and tax analytics")
print("   - Daily KPIs and trends")


In [41]:
# Verify data loaded
print("\n📊 Raw & Staging Data Summary:")
print(f"  CUSTOMER:     {session.table('RAW.RAW_CUSTOMER').count():,}")
print(f"  ORDERS:       {session.table('RAW.RAW_ORDERS').count():,}")
print(f"  LINEITEM:     {session.table('RAW.RAW_LINEITEM').count():,}")
print(f"  STG_CUSTOMER: {session.table('STAGING.STG_CUSTOMER').count():,}")
print(f"  STG_ORDERS:   {session.table('STAGING.STG_ORDERS').count():,}")
print(f"  STG_LINEITEM: {session.table('STAGING.STG_LINEITEM').count():,}")

print("\n✅ All data loaded directly from GitHub!")
print("\n📌 NEXT STEPS:")
print("   - Load remaining 3 batches (batch_002, batch_003, batch_004)")
print("   - Create Dynamic Tables (Bronze, Silver, Gold)")
print("   - Process incremental batches")
print("   - Compare with Databricks DLT results")

In [42]:
# Verify initial Dynamic Tables are created
print("📊 Dynamic Tables Status:")
print("=" * 80)

# Show all Dynamic Tables - use collect() to avoid column mismatch issues
dt_rows = session.sql("SHOW DYNAMIC TABLES IN DATABASE TPCH_DEMO").collect()

if dt_rows:
    print("\n🔄 Dynamic Tables:")
    print(f"{'Name':<30} {'Schema':<10} {'Target Lag':<15} {'Refresh Mode':<15} {'State':<15}")
    print("-" * 85)
    for row in dt_rows:
        print(f"{row['name']:<30} {row['schema_name']:<10} {row['target_lag']:<15} {row['refresh_mode']:<15} {row['scheduling_state']:<15}")
else:
    print("⚠️  No Dynamic Tables found")

# Check row counts in each layer
print("\n\n📊 Row Counts by Layer:")
print("=" * 80)

print("\n🥉 Bronze Layer:")
try:
    print(f"  BRONZE_CUSTOMER:  {session.table('BRONZE.BRONZE_CUSTOMER').count():,}")
    print(f"  BRONZE_ORDERS:    {session.table('BRONZE.BRONZE_ORDERS').count():,}")
    print(f"  BRONZE_LINEITEM:  {session.table('BRONZE.BRONZE_LINEITEM').count():,}")
except Exception as e:
    print(f"  ⚠️  Error: {e}")

print("\n🥈 Silver Layer:")
try:
    print(f"  SILVER_CUSTOMER:  {session.table('SILVER.SILVER_CUSTOMER').count():,}")
    print(f"  SILVER_ORDERS:    {session.table('SILVER.SILVER_ORDERS').count():,}")
    print(f"  SILVER_LINEITEM:  {session.table('SILVER.SILVER_LINEITEM').count():,}")
except Exception as e:
    print(f"  ⚠️  Error: {e}")

print("\n🥇 Gold Layer:")
try:
    print(f"  GOLD_CUSTOMER_ORDERS_SUMMARY:  {session.table('GOLD.GOLD_CUSTOMER_ORDERS_SUMMARY').count():,}")
    print(f"  GOLD_ORDER_LINEITEM_SUMMARY:   {session.table('GOLD.GOLD_ORDER_LINEITEM_SUMMARY').count():,}")
    print(f"  GOLD_DAILY_SALES_SUMMARY:      {session.table('GOLD.GOLD_DAILY_SALES_SUMMARY').count():,}")
except Exception as e:
    print(f"  ⚠️  Error: {e}")

print("\n\n✅ Complete! All Dynamic Tables created from GitHub data")
print("\n📌 Next Steps:")
print("   1. Monitor Dynamic Tables refresh: SHOW DYNAMIC TABLES")
print("   2. Load remaining batches (batch_002, batch_003, batch_004)")
print("   3. Verify incremental processing")
print("   4. Compare performance with Databricks DLT")


In [ ]:
# simulate CDC every 180 secs
import time

# use a different warehouse than what Dynamic Tables use isolate & track costs
session.use_warehouse("COMPUTE_WH")

def get_row_count(result):
    """Extracts the count from the first column of the first row of a collect() result."""
    try:
        # result is a list of Row objects; the first element [0] is the row,
        # and [0] of that row is the count value.
        return result[0][0]
    except (IndexError, TypeError):
        return 0

for batch_num in range(1, 5):
    batch_name = f"batch_{batch_num:03d}"
    
    print(f"\n📦 Processing {batch_name}...")
    
    # --- CUSTOMER ---
    res_cust_ins = session.sql(f"""
        INSERT INTO TPCH_DEMO.RAW.RAW_CUSTOMER
        (C_CUSTKEY, C_NAME, C_ADDRESS, C_NATIONKEY, C_PHONE, C_ACCTBAL, C_MKTSEGMENT, C_COMMENT)
        SELECT C_CUSTKEY, C_NAME, C_ADDRESS, C_NATIONKEY, C_PHONE, C_ACCTBAL, C_MKTSEGMENT, C_COMMENT
        FROM TPCH_DEMO.STAGING.STG_CUSTOMER
        WHERE CHANGE_TYPE = 'INSERT' AND BATCH_NUMBER = '{batch_name}'
    """).collect()
    print(f"  ✅ Customer: {get_row_count(res_cust_ins)} rows inserted")
    
    res_cust_upd = session.sql(f"""
        UPDATE TPCH_DEMO.RAW.RAW_CUSTOMER AS target
        SET C_NAME = source.C_NAME, C_ADDRESS = source.C_ADDRESS, C_NATIONKEY = source.C_NATIONKEY,
            C_PHONE = source.C_PHONE, C_ACCTBAL = source.C_ACCTBAL, C_MKTSEGMENT = source.C_MKTSEGMENT,
            C_COMMENT = source.C_COMMENT
        FROM TPCH_DEMO.STAGING.STG_CUSTOMER AS source
        WHERE target.C_CUSTKEY = source.C_CUSTKEY
          AND source.CHANGE_TYPE = 'UPDATE' AND source.BATCH_NUMBER = '{batch_name}'
    """).collect()
    print(f"  ✅ Customer: {get_row_count(res_cust_upd)} rows updated")
    
    # --- ORDERS ---
    res_ord_ins = session.sql(f"""
        INSERT INTO TPCH_DEMO.RAW.RAW_ORDERS
        (O_ORDERKEY, O_CUSTKEY, O_ORDERSTATUS, O_TOTALPRICE, O_ORDERDATE, O_ORDERPRIORITY, O_CLERK, O_SHIPPRIORITY, O_COMMENT)
        SELECT O_ORDERKEY, O_CUSTKEY, O_ORDERSTATUS, O_TOTALPRICE, O_ORDERDATE, O_ORDERPRIORITY, O_CLERK, O_SHIPPRIORITY, O_COMMENT
        FROM TPCH_DEMO.STAGING.STG_ORDERS
        WHERE CHANGE_TYPE = 'INSERT' AND BATCH_NUMBER = '{batch_name}'
    """).collect()
    print(f"  ✅ Orders: {get_row_count(res_ord_ins)} rows inserted")
    
    res_ord_upd = session.sql(f"""
        UPDATE TPCH_DEMO.RAW.RAW_ORDERS AS target
        SET O_CUSTKEY = source.O_CUSTKEY, O_ORDERSTATUS = source.O_ORDERSTATUS, O_TOTALPRICE = source.O_TOTALPRICE * 1.1,
            O_ORDERDATE = source.O_ORDERDATE, O_ORDERPRIORITY = source.O_ORDERPRIORITY, O_CLERK = source.O_CLERK,
            O_SHIPPRIORITY = source.O_SHIPPRIORITY, O_COMMENT = source.O_COMMENT
        FROM TPCH_DEMO.STAGING.STG_ORDERS AS source
        WHERE target.O_ORDERKEY = source.O_ORDERKEY
          AND source.CHANGE_TYPE = 'UPDATE' AND source.BATCH_NUMBER = '{batch_name}'
    """).collect()
    print(f"  ✅ Orders: {get_row_count(res_ord_upd)} rows updated")
    
    # --- LINEITEM ---
    res_line_ins = session.sql(f"""
        INSERT INTO TPCH_DEMO.RAW.RAW_LINEITEM
        (L_ORDERKEY, L_PARTKEY, L_SUPPKEY, L_LINENUMBER, L_QUANTITY, L_EXTENDEDPRICE, L_DISCOUNT, L_TAX, L_RETURNFLAG, L_LINESTATUS, L_SHIPDATE, L_COMMITDATE, L_RECEIPTDATE, L_SHIPINSTRUCT, L_SHIPMODE, L_COMMENT)
        SELECT L_ORDERKEY, L_PARTKEY, L_SUPPKEY, L_LINENUMBER, L_QUANTITY, L_EXTENDEDPRICE, L_DISCOUNT, L_TAX, L_RETURNFLAG, L_LINESTATUS, L_SHIPDATE, L_COMMITDATE, L_RECEIPTDATE, L_SHIPINSTRUCT, L_SHIPMODE, L_COMMENT
        FROM TPCH_DEMO.STAGING.STG_LINEITEM
        WHERE CHANGE_TYPE = 'INSERT' AND BATCH_NUMBER = '{batch_name}'
    """).collect()
    print(f"  ✅ Lineitem: {get_row_count(res_line_ins)} rows inserted")
    
    print(f"✅ {batch_name} processed successfully!")
    
    if batch_num < 4:
        print(f"⏳ Waiting 180 seconds before next batch...")
        time.sleep(180)

print(f"\n✅ All 4 batches (batch_001 to batch_004) processed successfully!")

In [64]:
-- Show Staging Tables


 SELECT 
    'ORDERS' as TABLE_NAME,
    BATCH_NUMBER,
    COUNT(CASE WHEN CHANGE_TYPE = 'INSERT' THEN 1 END) AS NO_OF_INSERTS,
    COUNT(CASE WHEN CHANGE_TYPE = 'UPDATE' THEN 1 END) AS NO_OF_UPDATES,
    COUNT(*) AS TOTAL_RECORDS
FROM TPCH_DEMO.STAGING.STG_ORDERS 
GROUP BY BATCH_NUMBER
--ORDER BY BATCH_NUMBER

UNION 

 SELECT 
    'LINEITEMS' as TABLE_NAME,
    BATCH_NUMBER,
    COUNT(CASE WHEN CHANGE_TYPE = 'INSERT' THEN 1 END) AS NO_OF_INSERTS,
    COUNT(CASE WHEN CHANGE_TYPE = 'UPDATE' THEN 1 END) AS NO_OF_UPDATES,
    COUNT(*) AS TOTAL_RECORDS
FROM TPCH_DEMO.STAGING.STG_LINEITEM 
GROUP BY BATCH_NUMBER
--ORDER BY BATCH_NUMBER

UNION 

 SELECT 
    'CUSTOMERS' as TABLE_NAME,
    BATCH_NUMBER,
    COUNT(CASE WHEN CHANGE_TYPE = 'INSERT' THEN 1 END) AS NO_OF_INSERTS,
    COUNT(CASE WHEN CHANGE_TYPE = 'UPDATE' THEN 1 END) AS NO_OF_UPDATES,
    COUNT(*) AS TOTAL_RECORDS
FROM TPCH_DEMO.STAGING.STG_CUSTOMER
GROUP BY BATCH_NUMBER
ORDER BY  BATCH_NUMBER, TABLE_NAME